In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

evaluating the schema

In [0]:
#option("header",True) will consider the first row as header
#option("inferSchema",True) will automatically or by itself decode the datatypes of the header
df1 = spark.read.options(inferSchema=True,header=True).csv("/FileStore/tables/StudentData-1.csv") 
df1.show(10)
df1.printSchema()

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC|52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP|61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud|72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC|81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP|92882|   51|Judie Chipps_Clem...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 10 

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema=StructType([
    StructField("age",IntegerType(),True),
    StructField("gender",StringType(),True),
    StructField("name",StringType(),True),
    StructField("course",StringType(),True),
    StructField("roll",StringType(),True),
    StructField("marks",IntegerType(),True),
    StructField("email",StringType(),True),
])

In [0]:
df = spark.read.options(header=True).schema(schema).csv("/FileStore/tables/StudentData-1.csv") 
df.show(10)
df.printSchema()

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC|52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP|61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud|72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC|81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP|92882|   51|Judie Chipps_Clem...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 10 

creating df from rdd

In [0]:
from pyspark import SparkConf,SparkContext
conf=SparkConf().setAppName("dataframes")
sc=SparkContext.getOrCreate(conf=conf)
rdd=sc.textFile("/FileStore/tables/StudentData-1.csv")
rdd.first()

Out[6]: 'age,gender,name,course,roll,marks,email'

In [0]:
# Get the header from the RDD
header = rdd.first()
rdd = rdd.filter(lambda x: x!=header).map(lambda x: x.split(','))
rdd = rdd.map(lambda x: [int(x[0]),x[1],x[2],x[3],x[4],int(x[5]),x[6]] ) #changing the data type to int
rdd.collect()


Out[7]: [[28,
  'Female',
  'Hubert Oliveras',
  'DB',
  '02984',
  59,
  'Annika Hoffman_Naoma Fritts@OOP.com'],
 [29,
  'Female',
  'Toshiko Hillyard',
  'Cloud',
  '12899',
  62,
  'Margene Moores_Marylee Capasso@DB.com'],
 [28,
  'Male',
  'Celeste Lollis',
  'PF',
  '21267',
  45,
  'Jeannetta Golden_Jenna Montague@DSA.com'],
 [29,
  'Female',
  'Elenore Choy',
  'DB',
  '32877',
  29,
  'Billi Clore_Mitzi Seldon@DB.com'],
 [28,
  'Male',
  'Sheryll Towler',
  'DSA',
  '41487',
  41,
  'Claude Panos_Judie Chipps@OOP.com'],
 [28,
  'Male',
  'Margene Moores',
  'MVC',
  '52771',
  32,
  'Toshiko Hillyard_Clementina Menke@MVC.com'],
 [28,
  'Male',
  'Neda Briski',
  'OOP',
  '61973',
  69,
  'Alberta Freund_Elenore Choy@DB.com'],
 [28,
  'Female',
  'Claude Panos',
  'Cloud',
  '72409',
  85,
  'Sheryll Towler_Alberta Freund@Cloud.com'],
 [28,
  'Male',
  'Celeste Lollis',
  'MVC',
  '81492',
  64,
  'Nicole Harwood_Claude Panos@MVC.com'],
 [29,
  'Male',
  'Cordie Harnois',
  'OOP

In [0]:
rdd1 = rdd.toDF()
rdd1.show(5) #this is without header

+---+------+----------------+-----+-----+---+--------------------+
| _1|    _2|              _3|   _4|   _5| _6|                  _7|
+---+------+----------------+-----+-----+---+--------------------+
| 28|Female| Hubert Oliveras|   DB|02984| 59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard|Cloud|12899| 62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|   PF|21267| 45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|   DB|32877| 29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|  DSA|41487| 41|Claude Panos_Judi...|
+---+------+----------------+-----+-----+---+--------------------+
only showing top 5 rows



In [0]:
dfrdd = spark.createDataFrame(rdd,schema=schema)
dfrdd.show()
dfrdd.printSchema()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 

selecting specific columns from df

In [0]:
df.select("age","name").show(5)

+---+----------------+
|age|            name|
+---+----------------+
| 28| Hubert Oliveras|
| 29|Toshiko Hillyard|
| 28|  Celeste Lollis|
| 29|    Elenore Choy|
| 28|  Sheryll Towler|
+---+----------------+
only showing top 5 rows



In [0]:
df.select(df.name,df.email).show(10)

+----------------+--------------------+
|            name|               email|
+----------------+--------------------+
| Hubert Oliveras|Annika Hoffman_Na...|
|Toshiko Hillyard|Margene Moores_Ma...|
|  Celeste Lollis|Jeannetta Golden_...|
|    Elenore Choy|Billi Clore_Mitzi...|
|  Sheryll Towler|Claude Panos_Judi...|
|  Margene Moores|Toshiko Hillyard_...|
|     Neda Briski|Alberta Freund_El...|
|    Claude Panos|Sheryll Towler_Al...|
|  Celeste Lollis|Nicole Harwood_Cl...|
|  Cordie Harnois|Judie Chipps_Clem...|
+----------------+--------------------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import col
df.select(col("roll"),col("name")).show(5)

+-----+----------------+
| roll|            name|
+-----+----------------+
|02984| Hubert Oliveras|
|12899|Toshiko Hillyard|
|21267|  Celeste Lollis|
|32877|    Elenore Choy|
|41487|  Sheryll Towler|
+-----+----------------+
only showing top 5 rows



In [0]:
df.select('*').show(5) #gives all the columns in the df

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



In [0]:
df.columns #gives the name of the columns

Out[14]: ['age', 'gender', 'name', 'course', 'roll', 'marks', 'email']

In [0]:
df.select(df.columns[:4]).show(5)

+---+------+----------------+------+
|age|gender|            name|course|
+---+------+----------------+------+
| 28|Female| Hubert Oliveras|    DB|
| 29|Female|Toshiko Hillyard| Cloud|
| 28|  Male|  Celeste Lollis|    PF|
| 29|Female|    Elenore Choy|    DB|
| 28|  Male|  Sheryll Towler|   DSA|
+---+------+----------------+------+
only showing top 5 rows



withColumn in df

In [0]:
from pyspark.sql.functions import col
df1 = df1.withColumn("roll",col("roll").cast("String")) #converting roll no to string using with column
df1.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



In [0]:
df1.withColumn("newmarks",col("marks")+10).show(5) #making a new column showing up with updated marks

+---+------+----------------+------+-----+-----+--------------------+--------+
|age|gender|            name|course| roll|marks|               email|newmarks|
+---+------+----------------+------+-----+-----+--------------------+--------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|      69|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|      72|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|      55|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|      39|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|      51|
+---+------+----------------+------+-----+-----+--------------------+--------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import lit
df1.withColumn("Country",lit("India")).show(5)

+---+------+----------------+------+-----+-----+--------------------+-------+
|age|gender|            name|course| roll|marks|               email|Country|
+---+------+----------------+------+-----+-----+--------------------+-------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|  India|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|  India|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|  India|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|  India|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|  India|
+---+------+----------------+------+-----+-----+--------------------+-------+
only showing top 5 rows



In [0]:
df1.withColumn("marks",col("marks")-10).withColumn("newmarks",col("marks")+20).show(5)

+---+------+----------------+------+-----+-----+--------------------+--------+
|age|gender|            name|course| roll|marks|               email|newmarks|
+---+------+----------------+------+-----+-----+--------------------+--------+
| 28|Female| Hubert Oliveras|    DB| 2984|   49|Annika Hoffman_Na...|      69|
| 29|Female|Toshiko Hillyard| Cloud|12899|   52|Margene Moores_Ma...|      72|
| 28|  Male|  Celeste Lollis|    PF|21267|   35|Jeannetta Golden_...|      55|
| 29|Female|    Elenore Choy|    DB|32877|   19|Billi Clore_Mitzi...|      39|
| 28|  Male|  Sheryll Towler|   DSA|41487|   31|Claude Panos_Judi...|      51|
+---+------+----------------+------+-----+-----+--------------------+--------+
only showing top 5 rows



withColumnRenamed and Alias

In [0]:
df1.withColumnRenamed("gender","sex").show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|   sex|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



In [0]:
df.select(col("name").alias("full name")).show(5) #to rename the column only while reading it thru

+----------------+
|       full name|
+----------------+
| Hubert Oliveras|
|Toshiko Hillyard|
|  Celeste Lollis|
|    Elenore Choy|
|  Sheryll Towler|
+----------------+
only showing top 5 rows



filter/where in df

In [0]:
df.filter(df.course == "DB").show(5)

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 29|  Male|Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|Female| Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 29|Female| Latia Vanhoose|    DB|152159|   27|Claude Panos_Sant...|
+---+------+---------------+------+------+-----+--------------------+
only showing top 5 rows



In [0]:
df.filter(col("course") == "DB").show(5)

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 29|  Male|Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|Female| Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 29|Female| Latia Vanhoose|    DB|152159|   27|Claude Panos_Sant...|
+---+------+---------------+------+------+-----+--------------------+
only showing top 5 rows



In [0]:
df.filter((df.course == "DB") & (df.marks < 50) ).show(5)
df.filter((df.course == "DB") & (df.marks < 50) ).count()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|Female|  Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 29|Female|  Latia Vanhoose|    DB|152159|   27|Claude Panos_Sant...|
| 28|  Male|   Kizzy Brenner|    DB|381712|   36|Paris Hutton_Kena...|
| 28|  Male|Toshiko Hillyard|    DB|392218|   47|Leontine Phillips...|
+---+------+----------------+------+------+-----+--------------------+
only showing top 5 rows

Out[24]: 56

In [0]:
courses = ["DB","Cloud","OOP","DSA"]
df.filter(df.course.isin(courses)).show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
| 28|  Male|     Neda Briski|   OOP|61973|   69|Alberta Freund_El...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



In [0]:
df.filter(df.course.startswith("D")).show(5)

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male| Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 29|Female|      Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29|  Male|Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
+---+------+---------------+------+------+-----+--------------------+
only showing top 5 rows



In [0]:
df.filter(df.course.endswith("A")).show(5)

+---+------+--------------+------+------+-----+--------------------+
|age|gender|          name|course|  roll|marks|               email|
+---+------+--------------+------+------+-----+--------------------+
| 28|  Male|Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 29|Female|     Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 28|Female|  Jc Andrepont|   DSA|232060|   58|Billi Clore_Abram...|
| 29|Female|  Paris Hutton|   DSA|271472|   99|Sheryll Towler_Al...|
| 28|Female|Dustin Feagins|   DSA|291984|   82|Abram Nagao_Kena ...|
+---+------+--------------+------+------+-----+--------------------+
only showing top 5 rows



In [0]:
df.filter(df.name.contains("ac")).show(5) #names with ac anywhere in between them are considered

+---+------+---------------+------+-------+-----+--------------------+
|age|gender|           name|course|   roll|marks|               email|
+---+------+---------------+------+-------+-----+--------------------+
| 29|  Male|Ernest Rossbach|    DB| 111449|   53|Maybell Duguay_Ab...|
| 29|Female|Ernest Rossbach|   MVC|1201427|   80|Elenore Choy_Jenn...|
| 29|Female|Ernest Rossbach| Cloud|1821977|   28|Annika Hoffman_Ki...|
| 28|  Male|Ernest Rossbach|    DB|1901492|   54|Latia Vanhoose_Jc...|
| 28|Female|Ernest Rossbach| Cloud|2602027|   83|Alberta Freund_Go...|
+---+------+---------------+------+-------+-----+--------------------+
only showing top 5 rows



In [0]:
df.filter(df.name.like('%a%e%')).show(5)

+---+------+--------------+------+------+-----+--------------------+
|age|gender|          name|course|  roll|marks|               email|
+---+------+--------------+------+------+-----+--------------------+
| 28|  Male|Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|Female|  Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|Female|Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 29|Female|Latia Vanhoose|   MVC|132110|   55|Eda Neathery_Nico...|
| 29|Female|Latia Vanhoose|    DB|152159|   27|Claude Panos_Sant...|
+---+------+--------------+------+------+-----+--------------------+
only showing top 5 rows



distinct, count, duplicate

In [0]:
df.distinct().count()

Out[30]: 1000

In [0]:
df.select("gender").distinct().show()

+------+
|gender|
+------+
|Female|
|  Male|
+------+



In [0]:
df.dropDuplicates(["gender","course"]).show()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29|Female|  Latia Vanhoose|   MVC|132110|   55|Eda Neathery_Nico...|
| 28|Female|  Alberta Freund|   OOP|251805|   83|Annika Hoffman_Sh...|
| 29|Female|  Loris Crossett|    PF|201487|   96|Elenore Choy_Lati...|
| 29|  Male|     Billi Clore| Cloud|512047|   76|Taryn Brownlee_Ju...|
| 29|  Male| Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28| 

sort/orderBy

In [0]:
df.sort("name").show(5)

+---+------+-----------+------+-------+-----+--------------------+
|age|gender|       name|course|   roll|marks|               email|
+---+------+-----------+------+-------+-----+--------------------+
| 29|Female|Abram Nagao|   DSA|2041248|   72|Latia Vanhoose_Ke...|
| 28|  Male|Abram Nagao|   DSA|3781521|   45|Taryn Brownlee_Mi...|
| 28|Female|Abram Nagao|    DB|2632057|   25|Margene Moores_So...|
| 29|  Male|Abram Nagao|   MVC| 962235|   32|Jenna Montague_Go...|
| 28|  Male|Abram Nagao| Cloud|2652463|   21|Eda Neathery_Anni...|
+---+------+-----------+------+-------+-----+--------------------+
only showing top 5 rows



In [0]:
df.sort("name","age").show(10) #first the data will be sorted on the basis of name then on age

+---+------+-----------+------+-------+-----+--------------------+
|age|gender|       name|course|   roll|marks|               email|
+---+------+-----------+------+-------+-----+--------------------+
| 28|  Male|Abram Nagao|   DSA|1382959|   66|Michelle Ruggiero...|
| 28|  Male|Abram Nagao|   DSA|3781521|   45|Taryn Brownlee_Mi...|
| 28|Female|Abram Nagao|    DB|2632057|   25|Margene Moores_So...|
| 28|  Male|Abram Nagao| Cloud|2652463|   21|Eda Neathery_Anni...|
| 28|Female|Abram Nagao| Cloud|2811189|   54|Kena Wild_Donna Y...|
| 28|Female|Abram Nagao| Cloud|5301859|   50|Elenore Choy_Clau...|
| 28|  Male|Abram Nagao|   OOP|5571082|   85|Santa Kerfien_Jal...|
| 29|  Male|Abram Nagao|   MVC| 962235|   32|Jenna Montague_Go...|
| 29|  Male|Abram Nagao|    PF|1112161|   42|Nicole Harwood_Ni...|
| 29|Female|Abram Nagao|   DSA|1181007|   57|Anna Santos_Anna ...|
+---+------+-----------+------+-------+-----+--------------------+
only showing top 10 rows



In [0]:
df.orderBy(df.age,df.marks).show(10)

+---+------+---------------+------+-------+-----+--------------------+
|age|gender|           name|course|   roll|marks|               email|
+---+------+---------------+------+-------+-----+--------------------+
| 28|Female| Maybell Duguay| Cloud| 261439|   20|Nicole Harwood_Ju...|
| 28|Female|   Jc Andrepont|    PF| 972733|   20|Eda Neathery_Eda ...|
| 28|  Male|Marylee Capasso|   DSA|2081560|   20|Sheryll Towler_Do...|
| 28|  Male|    Abram Nagao| Cloud|2652463|   21|Eda Neathery_Anni...|
| 28|  Male|  Kizzy Brenner| Cloud|4622328|   21|Leontine Phillips...|
| 28|Female|Sebrina Maresca|    PF|5742239|   21|Bonita Higuera_Se...|
| 28|  Male| Bonita Higuera| Cloud|7671835|   21|Taryn Brownlee_Se...|
| 28|  Male|    Billi Clore|    DB|9151306|   21|Latia Vanhoose_Er...|
| 28|Female|   Cheri Kenney| Cloud|2281771|   22|Tamera Blakley_Ab...|
| 28|Female|    Donna Yerby| Cloud|4392575|   22|Leontine Phillips...|
+---+------+---------------+------+-------+-----+--------------------+
only s

In [0]:
df.sort(df.marks.asc(),df.age.desc()).show(10) #same can be done with orderby

+---+------+-----------------+------+-------+-----+--------------------+
|age|gender|             name|course|   roll|marks|               email|
+---+------+-----------------+------+-------+-----+--------------------+
| 29|  Male|     Elenore Choy|    DB|3652057|   20|Jc Andrepont_Gonz...|
| 29|Female|   Tamera Blakley|   DSA|3911247|   20|Donna Yerby_Bonit...|
| 29|  Male|   Jalisa Swenson|   OOP|4751515|   20|Annika Hoffman_Hu...|
| 29|Female|  Gonzalo Ferebee|   DSA|5631172|   20|Jeannetta Golden_...|
| 29|  Male|Michelle Ruggiero|    PF|6001585|   20|Paris Hutton_Marg...|
| 29|Female|Michelle Ruggiero|    DB|9232210|   20|Donna Yerby_Latia...|
| 28|Female|   Maybell Duguay| Cloud| 261439|   20|Nicole Harwood_Ju...|
| 28|Female|     Jc Andrepont|    PF| 972733|   20|Eda Neathery_Eda ...|
| 28|  Male|  Marylee Capasso|   DSA|2081560|   20|Sheryll Towler_Do...|
| 29|Female|  Gonzalo Ferebee|   OOP|2262603|   21|Bonita Higuera_Ch...|
+---+------+-----------------+------+-------+-----+

In [0]:
df.groupBy.show()
#group by cannot be directly shown coz it needs an external extra parameter to show it with because it doesnt perform the aggregation, it just prepares the data for aggregation

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-2177976991470628>:1
----> 1 df.groupBy.show()

AttributeError: 'function' object has no attribute 'show'

In [0]:
from pyspark.sql.functions import sum,min,max,avg,mean,count
df.groupBy("gender").sum().show()

+------+--------+----------+
|gender|sum(age)|sum(marks)|
+------+--------+----------+
|Female|   14273|     29636|
|  Male|   14233|     30461|
+------+--------+----------+



In [0]:
df.groupBy("gender").count().show()
df.groupBy("course").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|  501|
|  Male|  499|
+------+-----+

+------+-----+
|course|count|
+------+-----+
|    PF|  166|
|    DB|  157|
|   MVC|  157|
|   DSA|  176|
| Cloud|  192|
|   OOP|  152|
+------+-----+



In [0]:
df.groupBy("gender").max("marks").show()
df.groupBy("gender").min("marks").show()

+------+----------+
|gender|max(marks)|
+------+----------+
|Female|        99|
|  Male|        99|
+------+----------+

+------+----------+
|gender|min(marks)|
+------+----------+
|Female|        20|
|  Male|        20|
+------+----------+



In [0]:
df.groupBy("age").avg("marks").show()

+---+------------------+
|age|        avg(marks)|
+---+------------------+
| 28|60.487854251012145|
| 29|59.715415019762844|
+---+------------------+



In [0]:
df.groupBy("gender").mean("marks").show()

+------+------------------+
|gender|        avg(marks)|
+------+------------------+
|Female|59.153692614770456|
|  Male| 61.04408817635271|
+------+------------------+



In [0]:
df.groupBy("course","gender").count().show()

+------+------+-----+
|course|gender|count|
+------+------+-----+
|   OOP|  Male|   70|
|    DB|  Male|   82|
| Cloud|Female|  106|
|   MVC|  Male|   86|
|   DSA|Female|   98|
|    PF|  Male|   97|
|   MVC|Female|   71|
| Cloud|  Male|   86|
|    PF|Female|   69|
|   DSA|  Male|   78|
|    DB|Female|   75|
|   OOP|Female|   82|
+------+------+-----+



In [0]:
df.groupBy("course","gender").agg(count("*").alias("total_enrollments"), sum("marks").alias("total_marks"), min("marks").alias("min_makrs"), max("marks"), avg("marks")).show()


+------+------+-----------------+-----------+---------+----------+------------------+
|course|gender|total_enrollments|total_marks|min_makrs|max(marks)|        avg(marks)|
+------+------+-----------------+-----------+---------+----------+------------------+
|   OOP|  Male|               70|       4234|       20|        99| 60.48571428571429|
|    DB|  Male|               82|       5073|       20|        98| 61.86585365853659|
| Cloud|Female|              106|       6316|       20|        99| 59.58490566037736|
|   MVC|  Male|               86|       5241|       22|        99| 60.94186046511628|
|   DSA|Female|               98|       6124|       20|        99| 62.48979591836735|
|    PF|  Male|               97|       5960|       20|        99| 61.44329896907217|
|   MVC|Female|               71|       4344|       22|        99|61.183098591549296|
| Cloud|  Male|               86|       5127|       21|        97|59.616279069767444|
|    PF|Female|               69|       3973|       20

In [0]:
df.filter(df.gender == "Male").groupBy("course","gender").agg(count('*')).show()

+------+------+--------+
|course|gender|count(1)|
+------+------+--------+
|   OOP|  Male|      70|
|    DB|  Male|      82|
|   MVC|  Male|      86|
|    PF|  Male|      97|
| Cloud|  Male|      86|
|   DSA|  Male|      78|
+------+------+--------+



In [0]:
df.filter(df.gender == "Male").groupBy("course","gender").agg(count('*').alias("enrollments")).show() 
#count is renamed as enrollments

+------+------+-----------+
|course|gender|enrollments|
+------+------+-----------+
|   OOP|  Male|         70|
|    DB|  Male|         82|
|   MVC|  Male|         86|
|    PF|  Male|         97|
| Cloud|  Male|         86|
|   DSA|  Male|         78|
+------+------+-----------+



In [0]:
df.filter(df.gender == "Male").groupBy("course","gender").agg(count('*').alias("enrollments")).filter(col("enrollments") > 85).show()


+------+------+-----------+
|course|gender|enrollments|
+------+------+-----------+
|   MVC|  Male|         86|
|    PF|  Male|         97|
| Cloud|  Male|         86|
+------+------+-----------+



user defined function

In [0]:
df1 = df1.withColumn("BonusMarks",lit(20))
df1.show(5)

+---+------+----------------+------+-----+-----+--------------------+----------+
|age|gender|            name|course| roll|marks|               email|BonusMarks|
+---+------+----------------+------+-----+-----+--------------------+----------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|        20|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|        20|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|        20|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|        20|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|        20|
+---+------+----------------+------+-----+-----+--------------------+----------+
only showing top 5 rows



In [0]:
df1.columns

Out[49]: ['age', 'gender', 'name', 'course', 'roll', 'marks', 'email', 'BonusMarks']

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
def get_total_marks(course,marks,BonusMarks):
    if course == "DB":
        return marks+BonusMarks
    elif course == "DSA":
        return marks-BonusMarks
    else:
        return marks
marksudf = udf(lambda x,y,z: get_total_marks(x,y,z),IntegerType())
df1.withColumn("newmarks",marksudf(df1.course,df1.marks,df1.BonusMarks)).show(5)

+---+------+----------------+------+-----+-----+--------------------+----------+--------+
|age|gender|            name|course| roll|marks|               email|BonusMarks|newmarks|
+---+------+----------------+------+-----+-----+--------------------+----------+--------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|        20|      79|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|        20|      62|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|        20|      45|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|        20|      49|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|        20|      21|
+---+------+----------------+------+-----+-----+--------------------+----------+--------+
only showing top 5 rows



Cache and Persist - to save the data temporarily in the memory to optimise the workflow

In [0]:
df1 = df1.drop(df1.BonusMarks)

In [0]:
df1.count()

Out[54]: 1000

In [0]:
df1.cache()

Out[56]: DataFrame[age: int, gender: string, name: string, course: string, roll: string, marks: int, email: string]

In [0]:
df1.show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



df to rdd

In [0]:
type(df)

Out[60]: pyspark.sql.dataframe.DataFrame

In [0]:
rdd = df.rdd

In [0]:
type(rdd)

Out[62]: pyspark.rdd.RDD

In [0]:
rdd.filter(lambda x:x["marks"]==20).collect()

Out[77]: [Row(age=28, gender='Female', name='Maybell Duguay', course='Cloud', roll='261439', marks=20, email='Nicole Harwood_Judie Chipps@DB.com'),
 Row(age=28, gender='Female', name='Jc Andrepont', course='PF', roll='972733', marks=20, email='Eda Neathery_Eda Neathery@MVC.com'),
 Row(age=28, gender='Male', name='Marylee Capasso', course='DSA', roll='2081560', marks=20, email='Sheryll Towler_Donna Yerby@Cloud.com'),
 Row(age=29, gender='Male', name='Elenore Choy', course='DB', roll='3652057', marks=20, email='Jc Andrepont_Gonzalo Ferebee@DB.com'),
 Row(age=29, gender='Female', name='Tamera Blakley', course='DSA', roll='3911247', marks=20, email='Donna Yerby_Bonita Higuera@DB.com'),
 Row(age=29, gender='Male', name='Jalisa Swenson', course='OOP', roll='4751515', marks=20, email='Annika Hoffman_Hubert Oliveras@Cloud.com'),
 Row(age=29, gender='Female', name='Gonzalo Ferebee', course='DSA', roll='5631172', marks=20, email='Jeannetta Golden_Hubert Oliveras@MVC.com'),
 Row(age=29, gender='M

Spark SQL

In [0]:
df.createOrReplaceTempView("Student") #registering the df as a table named Student

In [0]:
spark.sql("select name from Student where marks=20").show()

+-----------------+
|             name|
+-----------------+
|   Maybell Duguay|
|     Jc Andrepont|
|  Marylee Capasso|
|     Elenore Choy|
|   Tamera Blakley|
|   Jalisa Swenson|
|  Gonzalo Ferebee|
|Michelle Ruggiero|
|Michelle Ruggiero|
+-----------------+



In [0]:
#different available modes:
#overwrite - deletes existing data and writes new data over it
#ignore - writes the data only if the file doesnt exist already
#append - adds new data to existing dataset
#error - raises an error if the file exists

In [0]:
df.write.mode('overwrite').options(header='True').csv('/FileStore/tables/Student')